In [1]:
import argparse
from config import Config
from restCall import HLData

config = Config('/work/Decks/KslawElevate/config.json')
hl_data = HLData(config)


2022-06-02 12:58:11,841 [INFO] Collecting Highlight data for harness


KeyError: 'Application not found: harness'

In [ ]:

hl_id = 'Concord'
cve_df = hl_data.get_cve_data(hl_id)
cve_df.sort_values(['cweId','cve'], ascending=True, inplace=True)
cwe_df = cve_df.groupby(['cweId'])['cve'].apply(lambda x: ', '.join(x)).reset_index()


In [4]:
cve_df = cve_df[['cve','cweId','criticity']].drop_duplicates()
critical_df = cve_df[cve_df['criticity']=='CRITICAL'].groupby(['cweId']).size().reset_index(name='Critical')
high_df = cve_df[cve_df['criticity']=='HIGH'].groupby(['cweId']).size().reset_index(name='High')
medium_df = cve_df[cve_df['criticity']=='MEDIUM'].groupby(['cweId']).size().reset_index(name='Medium')


In [5]:

final_df = critical_df.merge(high_df, left_on='cweId', right_on='cweId',how='outer').fillna(0)
final_df = final_df.merge(medium_df, left_on='cweId', right_on='cweId',how='outer').fillna(0)
final_df = final_df.merge(cwe_df, left_on='cweId', right_on='cweId',how='outer')
#
final_df['Total CVEs'] = final_df['Critical'] + final_df['High'] + final_df['Medium']
final_df = final_df[['cweId','cve','Critical','High','Medium','Total CVEs']].fillna(0)
final_df['cve']=final_df['cve'].str.replace('CVE-','')
final_df['cweId']=final_df['cweId'].str.replace('CWE-','')


In [2]:
from pandas import read_csv

cwe_top_25 = read_csv('cwe-top25.csv')
cwe_top_25['CWE-ID']='CWE-'+cwe_top_25['CWE-ID'].astype(str)
cwe_top_25=cwe_top_25[['CWE-ID','Name','Description','Extended Description','Common Consequences','Potential Mitigations']]

#final_df = final_df.merge(cwe_df, left_on='cweId', right_on='cweId',how='outer')

In [ ]:
import util
import pandas as pd

file_name = f'{config.output}/risk.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
col_widths=[50,10,10,10,10]
cloud_tab = util.format_table(writer,final_df,'Risk',col_widths)
writer.save()
